In [1]:
import pandas as pd
import numpy as np
import openpyxl

In [2]:
df1 = pd.read_excel('./data/카카오_시세추이.xlsx')

C:\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
# ATR(Average True Range) 변동성 지표 생성

df_atr = df1.iloc[:,[0,1,5,6]]
df_atr.columns = ['date', 'fin', 'high', 'low']

In [4]:
df_atr

,date,fin,high,low
0,2021/08/31,155000,155000,151000
1,2021/08/30,151500,153000,150500
2,2021/08/27,150000,150000,147000
3,2021/08/26,149500,152500,147500
4,2021/08/25,151000,153500,148500
...,...,...,...,...
2460,2011/09/06,134000,137800,130000
2461,2011/09/05,133800,136500,131000
2462,2011/09/02,134500,134500,127500
2463,2011/09/01,128000,129700,126000


In [5]:
lst = []
atr = []

for i in range(0,len(df_atr)-1):
    a = df_atr.iloc[i,2] - df_atr.iloc[i,3]
    b = df_atr.iloc[i,2] - df_atr.iloc[i+1,1]
    c = df_atr.iloc[i,3] - df_atr.iloc[i+1,1]
    lst = [abs(a),abs(b),abs(c)]
    atr.append(max(lst))

In [6]:
# y값(종가 데이터) 준비

fin = list(df_atr[0:-1]['fin'])
del fin[96:99] # 거래정지 시점 데이터 행 삭제

In [7]:
# 거래량 데이터 추출

df1 = df1.drop(len(df1)-1,axis=0)
df1 = df1[['일자', '거래량']]

In [8]:
# PER, PBR 추출

df2 = pd.read_excel('./data/카카오_PER.xlsx')
df2 = df2[['일자', 'PER', 'PBR']]

In [9]:
# 거래주체별 순매수 데이터 추출

df3 = pd.read_excel('./data/카카오_거래실적.xlsx')
df3 = df3.drop(columns=['전체'], axis=1)

In [10]:
# 데이터 병합
kakao = pd.merge(df1, df2, on='일자', how='outer')
kakao = pd.merge(kakao, df3, on='일자', how='outer')
kakao['ATR'] = atr

In [12]:
# 액면분할에 따른 전처리
kakao = kakao.drop(index = [96,97,98], axis=1)
kakao.iloc[96:,8] = kakao.iloc[96:,8]/5
kakao.iloc[96:,1] = kakao.iloc[96:,1]*5

In [13]:
# 날짜 데이터로 변환
kakao['일자'] = pd.to_datetime(kakao['일자'])
kakao.rename(columns={'일자':'DATE'}, inplace=True)

In [14]:
# 거시경제 지표와 병합
macro_data = pd.read_csv('./data/merge_data.csv')
macro_data['DATE'] = pd.to_datetime(macro_data['DATE'])    # 날짜 데이터로 변환
macro_data = macro_data.dropna(axis=0)                     # 결측치가 포함된 행 제거
df_kakao = pd.merge(kakao, macro_data, how='left', on='DATE')

In [15]:
df_kakao

,DATE,거래량,PER,PBR,기관 합계,기타법인,개인,외국인 합계,ATR,NASDAQ,S&P,CBOE,Exchange rate,futures2y,futures10y
0,2021-08-31,3078357,420.05,10.85,6.959690e+09,1.378349e+09,-1.612465e+11,1.529085e+11,4000.0,"15,259.20","4,522.68",16.48,"1,158.73",110.32,134.05
1,2021-08-30,1628144,410.57,10.60,1.255289e+10,1.316290e+09,-3.743141e+10,2.356223e+10,3000.0,"15,265.90","4,528.79",16.19,"1,164.34",110.32,134.13
2,2021-08-27,1430634,406.5,10.50,2.713487e+10,1.398185e+08,-2.124092e+10,-6.033772e+09,3000.0,"15,129.50","4,509.37",16.39,"1,161.23",110.30,133.89
3,2021-08-26,2360164,405.15,10.46,5.346560e+08,-6.785710e+08,1.791563e+10,-1.777172e+10,5000.0,"14,945.80","4,469.91",18.84,"1,170.53",110.25,133.59
4,2021-08-25,2900928,409.21,10.57,1.294530e+10,-3.223110e+09,-6.005923e+10,5.033704e+10,5000.0,"15,041.90","4,496.19",16.79,"1,163.87",110.25,133.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2456,2011-09-07,803630,16.21,5.20,4.336654e+09,-1.527227e+08,-3.492414e+09,-6.915175e+08,1280.0,"2,548.90","1,198.62",33.38,"1,069.65",110.27,130.89
2457,2011-09-06,968495,15.88,5.10,2.520384e+09,1.636744e+08,-2.269462e+09,-4.145962e+08,1560.0,"2,473.80","1,165.24",37.00,"1,074.65",110.27,131.22
2458,2011-09-05,990510,15.86,5.09,9.441809e+09,3.553339e+08,-1.896291e+09,-7.900851e+09,1100.0,NaN,NaN,NaN,NaN,NaN,NaN
2459,2011-09-02,1577340,15.94,5.12,1.038012e+10,-4.997355e+08,-2.446876e+09,-7.433510e+09,1400.0,"2,480.30","1,173.97",33.92,"1,065.60",110.25,131.30


In [16]:
# y값(종가 데이터) 병합
df_kakao['y'] = fin

In [17]:
# 종가 열 액면분할 반영
df_kakao.iloc[96:,15] = df_kakao.iloc[96:,15]/5

In [18]:
# 결측치를 전일 값으로 대체
df_kakao.sort_values('DATE', inplace=True)
df_kakao.fillna(method='ffill', inplace=True)

In [19]:
df_kakao.to_excel('./data/kakao.xlsx',index=False)

In [20]:
kakao_df = pd.read_excel('./data/kakao.xlsx')

In [21]:
kakao_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2461 entries, 0 to 2460
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   DATE           2461 non-null   datetime64[ns]
 1   거래량            2461 non-null   int64         
 2   PER            2461 non-null   object        
 3   PBR            2461 non-null   float64       
 4   기관 합계          2461 non-null   int64         
 5   기타법인           2461 non-null   int64         
 6   개인             2461 non-null   int64         
 7   외국인 합계         2461 non-null   int64         
 8   ATR            2461 non-null   int64         
 9   NASDAQ         2461 non-null   object        
 10  S&P            2461 non-null   object        
 11  CBOE           2461 non-null   float64       
 12  Exchange rate  2461 non-null   object        
 13  futures2y      2461 non-null   float64       
 14  futures10y     2461 non-null   float64       
 15  y              2461 n